In [5]:
import pathlib
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)


02/20/2023 13:36:17 - INFO - root -   Using localhost sqlite as the database backend. as Database not configured. Add a qa_config.py file in the Python path with DATABASE_URL set.Continuing with the default sqlite on localhost.


RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
the current application. To solve this, set up an application context
with app.app_context(). See the documentation for more information.

In [ ]:
# Configure the logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

# https://haystack.deepset.ai/tutorials/01_basic_qa_pipeline

Haystack is an end-end production ready NLP solution allowing rapid development with popular modules from Huggingface. In this tutorial we will cover the key components of haystack and how to interact with it. 

Haystack itself provides the relevant components to handle the data, pass it into a text pipeline, retrain the model, and interact with the model object.

## Data Handling

For data handling, Haystack uses the _DocumentStore_ module. Here the _DocumentStore_ acts as a database that stores the text and meta data in a format that can be rapidly queried when needing to retrieve the data for the model. There are multiple types of document stores such as _Elasticsearch_, FAISS (Meta's ), OpenSearch, SQL and a few more. If you want to learn more about the document stores capability you should check out the [Haystack documentation](https://docs.haystack.deepset.ai/docs/document_store). To integrate the _DocumentStore_, it needs to be initialized first. In the following example we will be applying the in memor _DocumentStore_ which is the simplest form of it.

In [6]:
from haystack.document_stores import InMemoryDocumentStore

ModuleNotFoundError: No module named 'haystack'